In [ ]:
import logging
import sys

# Configure logging to show DEBUG level messages
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
)

# Set logger for chemrxiv to DEBUG
chemrxiv_logger = logging.getLogger("chemrxiv")
chemrxiv_logger.setLevel(logging.DEBUG)

import chemrxiv

# Create a client with a smaller delay to speed up testing
client = chemrxiv.Client(delay_seconds=0.5)

# Test a simple search
print("\nTesting search for 'catalysis'")
search = chemrxiv.Search(
    term="catalysis", limit=3, sort=chemrxiv.SortCriterion.PUBLISHED_DATE_DESC
)

try:
    # Use list() to force evaluation of the generator
    results = list(client.results(search))
    print(f"Found {len(results)} results:")

    for i, result in enumerate(results, 1):
        print(f"{i}. {result.title}")
        print(f"   ID: {result.id}")
        print(f"   DOI: {result.doi}")
        print(
            f"   Authors: {', '.join(str(author) for author in result.authors)}"
        )
        print(f"   Published: {result.published_date}")
        print()

        # Let's save the first paper ID for testing item retrieval
        if i == 1:
            paper_id = result.id
            paper_doi = result.doi
except Exception as e:
    print(f"Error in search: {e}")

# Test getting a specific paper by ID
if "paper_id" in locals():
    print(f"\nTesting get paper by ID: {paper_id}")
    try:
        paper = client.item(paper_id)
        print(f"Found paper: {paper.title}")
        print(f"Authors: {', '.join(str(author) for author in paper.authors)}")
        print(f"Abstract: {paper.abstract[:100]}...")
    except Exception as e:
        print(f"Error getting paper by ID: {e}")

# Test getting a paper by DOI
if "paper_doi" in locals():
    print(f"\nTesting get paper by DOI: {paper_doi}")
    try:
        paper = client.item_by_doi(paper_doi)
        print(f"Found paper: {paper.title}")
        print(f"ID: {paper.id}")
        print(f"Abstract: {paper.abstract[:100]}...")
    except Exception as e:
        print(f"Error getting paper by DOI: {e}")

In [ ]:
"""
Examples for using the ChemRxiv API wrapper
"""

import logging

import chemrxiv
from chemrxiv.exceptions import HTTPError, ItemWithdrawnError

# Set up logging to see what's happening
logging.basicConfig(level=logging.INFO)

# Create a client
client = chemrxiv.Client()

# Example 1: Search for papers with a specific term
print("Example 1: Search for papers")
search = chemrxiv.Search(
    term="catalysis", limit=5, sort=chemrxiv.SortCriterion.PUBLISHED_DATE_DESC
)

# Get the results
try:
    results = list(client.results(search))
    print(f"Found {len(results)} results")
    for i, result in enumerate(results, 1):
        print(
            f"{i}. {result.title} by {', '.join(str(author) for author in result.authors)}"
        )
        print(f"   Published: {result.published_date}")
        print(f"   DOI: {result.doi}")
        print(
            f"   Categories: {', '.join(str(cat) for cat in result.categories)}"
        )
        print()
except Exception as e:
    print(f"Error in search: {e}")

# Example 2: Get all available categories
print("\nExample 2: Get all categories")
categories = client.categories()
print(f"Found {len(categories)} categories")
for i, category in enumerate(categories[:5], 1):  # Just print the first 5
    print(f"{i}. {category.name} ({category.id})")
    if category.description:
        print(f"   Description: {category.description}")
    if category.count:
        print(f"   Count: {category.count}")
    print()

# Example 3: Get all available licenses
print("\nExample 3: Get all licenses")
licenses = client.licenses()
print(f"Found {len(licenses)} licenses")
for i, license in enumerate(licenses, 1):
    print(f"{i}. {license.name} ({license.id})")
    if license.url:
        print(f"   URL: {license.url}")
    print()

# Example 4: Find a specific paper by ID
print("\nExample 4: Get a paper by ID")
# Use a real ID from the search results
paper_id = (
    "6826a61850018ac7c5739260"  # Use one of the real paper IDs from the search
)
try:
    paper = client.item(paper_id)
    print(f"Title: {paper.title}")
    print(f"Authors: {', '.join(str(author) for author in paper.authors)}")
    print(
        f"Abstract: {paper.abstract[:100]}..."
    )  # First 100 chars of abstract
    print(f"DOI: {paper.doi}")
except HTTPError as e:
    print(f"Error: {e}")
except ItemWithdrawnError:
    print("Error: Item has been withdrawn")
except Exception as e:
    print(f"Error: {e}")

# Example 5: Find a paper by DOI
print("\nExample 5: Get a paper by DOI")
# Use a real DOI from the search results
doi = (
    "10.26434/chemrxiv-2025-5j4tn"  # Use the DOI from the first search result
)
try:
    paper = client.item_by_doi(doi)
    print(f"Title: {paper.title}")
    print(f"Authors: {', '.join(str(author) for author in paper.authors)}")
    print(f"Abstract: {paper.abstract[:100]}...")
    print(f"ID: {paper.id}")
except HTTPError as e:
    print(f"Error: {e}")
except ItemWithdrawnError:
    print("Error: Item has been withdrawn")
except Exception as e:
    print(f"Error: {e}")

# Example 6: Search with filters
print("\nExample 6: Search with filters")
# Use real category IDs
filtered_search = chemrxiv.Search(
    term="catalysis",
    limit=3,
    search_date_from="2023-01-01T00:00:00.000Z",
    search_date_to="2025-05-19T00:00:00.000Z",  # Use current date
    category_ids=["605c72ef153207001f6470d4"],  # "Catalysis" category ID
)

try:
    filtered_results = list(client.results(filtered_search))
    print(f"Found {len(filtered_results)} filtered results")
    for i, result in enumerate(filtered_results, 1):
        print(f"{i}. {result.title}")
        print(f"   Published: {result.published_date}")
except Exception as e:
    print(f"Error: {e}")

# Example 7: Download a PDF
print("\nExample 7: Download a PDF")
if results and results[0].pdf_url:
    print(f"Downloading PDF for: {results[0].title}")
    try:
        path = results[0].download_pdf(
            filename=f"chemrxiv_paper_{results[0].id}.pdf"
        )
        print(f"Downloaded to: {path}")
    except Exception as e:
        print(f"Error downloading PDF: {e}")
else:
    print("No results with PDF URL available")